In [1]:
cd ..

/home/agrigorev/notebooks/home-depot/homedepot


In [2]:
import Google_spell_check

In [3]:
import numpy as np
import pandas as pd

In [15]:
from time import time

In [66]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.grid_search import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

from nltk.stem.porter import PorterStemmer
import re

In [5]:
stemmer = PorterStemmer()

In [87]:
root_path = '/home/agrigorev/notebooks/home-depot/input'

df_train = pd.read_csv(root_path + '/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(root_path + '/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv(root_path + '/product_descriptions.csv')
df_attr = pd.read_csv(root_path + '/attributes.csv', encoding='utf-8')

In [88]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
num_train = df_train.shape[0]

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [39]:
spell_check = Google_spell_check.spell_check_dict

In [40]:
print len(spell_check.keys())
print len(df_all.search_term)
print df_all.search_term.isin(set(spell_check.keys())).sum()


3364
240760
27812


In [42]:
def get_or_identity(query):
    return spell_check.get(query, query)

df_all.search_term = df_all.search_term.apply(get_or_identity)

In [43]:
from unidecode import unidecode


In [53]:
def str_stem(s): 
    if not isinstance(s, (str, unicode)):
        return []

    if isinstance(s, str):
        s = unicode(s.decode('utf-8'))

    # some title edits?
    s = s.replace("&quot;"," ")
    s = s.replace(u"è_"," ")
    s = s.replace(u"å¡"," ")
    s = s.replace(u"Û"," ")
    s = s.replace(u"åÊ"," ")
    s = s.replace(u"ÛÒ"," ")
    s = s.replace(u"Ûª"," ")
    s = s.replace(u"ÛÜ"," ")
    s = s.replace(u"Û÷"," ")
    s = s.replace(u"ÈÀ"," ")
    s = s.replace(u"ã¢"," ")        
    s = s.replace(u"Ã¥Â¡"," ")
    s = s.replace(u"ã¨_"," ")

    s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A?
    s = s.replace("U.S."," US ")
    

    s = s.lower()
    
    s = s.replace("deckover","deck over")
    s = s.replace("clouverleaf","cloverleaf")
    s = s.replace("grilll","gril")
    s = s.replace("rainbird","rain bird")
    s = s.replace("flexlock","flex lock")
    s = s.replace("ultratouch","ultra touch")
    s = s.replace("shelfes","shelfs")
    s = s.replace("shelfing","selfing")
    s = s.replace("kitchenen","kitchen")
    s = s.replace("boltless","bolt less")
    s = s.replace("drilll","drill")
    s = s.replace("ridgid","rigid")
    s = s.replace("emberglow","ember glow")
    s = s.replace("tooliet","toilet")
    s = s.replace("wiremesh","wire mesh")
    s = s.replace("prefinished","pre finished")
    s = s.replace("prehung","pre hung")
    s = s.replace("priceepfister","pfister")
    s = s.replace("ceilingg","ceiling")
    s = s.replace("trafficmaster","traffic master")
    s = s.replace("glacierr","glacier")
    s = s.replace("toolet","toilet")
    s = s.replace("watter","water")
    s = s.replace("rerefrigerator","refrigerator")
    s = s.replace("garagee","garage")
    s = s.replace("air condit","air conditioner")
    s = s.replace("deckpaint","deck paint")
    s = s.replace("drilll","drill")
    s = s.replace("flangee","flange")
    s = s.replace("chipaper","chipper")
    s = s.replace("cleanerr","cleaner")
    s = s.replace("fiberglasss","fiberglass")
    s = s.replace("paccage","package")
    s = s.replace("garagee","garage")
    s = s.replace("collegege","college")
    s = s.replace("whiteplywood","white plywood")
    s = s.replace("beadboard","breadboard")
    s = s.replace("porcelaine","porcelain")
    s = s.replace("treatedd","treated")
    s = s.replace("cedarsafe","cedar safe")
    s = s.replace("plywooden","plywood")
    s = s.replace("sundstrom","sandstorm")
    s = s.replace("sprkinler","sprinkler")
    s = s.replace("sinktop","sink top")
    s = s.replace("ourdoor","outdoor")
    s = s.replace("ultratouch","ultra touch")
    s = s.replace("decoart","deco art")
    s = s.replace("outdoorlounge","outdoor lounge")
    s = s.replace("outdoorfurniture","outdoor furniture")
    s = s.replace("pricepfister","pfister")
    s = s.replace("glaciar","glacier")
    s = s.replace("glacie","glacier")
    s = s.replace("tiolet","toilet")
    s = s.replace("handycap","handicap")
    s = s.replace("wayer","water")
    s = s.replace("undercounter","under counter")
    s = s.replace("basemetnt","basement")
    s = s.replace("rustollum","rustoleum")
    s = s.replace("heaterconditioner","air conditioner")
    s = s.replace("spliter","splitter")
    s = s.replace("berh","behr")
    s = s.replace("snow thower","snow blower")
    s = s.replace("powertool","power tool")
    s = s.replace("repir","repair")
    s = s.replace("condtioners","conditioners")
    s = s.replace("pannels","panels")
    s = s.replace("frostking","frost king")
    s = s.replace("flourescent","fluorescent")
    s = s.replace("closetmade","closetmaid")
    s = s.replace("repir","repair")
    s = s.replace("greecianmarble","grecian marble")
    s = s.replace("porcelin","porcelain")
    s = s.replace("flushmount","flush mount")
    s = s.replace("foof","foot")
    s = s.replace("incide","inside")
    s = s.replace("pedistal","pedestal")
    s = s.replace("miricale","miracle")
    s = s.replace("windos","windows")
    s = s.replace("closetmaid","closet maid")
    s = s.replace("aspiradora","aspirator")
    s = s.replace("bentgrass","bentgrass")
    s = s.replace("hindges","hinges")
    s = s.replace("hieght","height")
    s = s.replace("clab","clay")
    s = s.replace("procelian","porcelain")
    s = s.replace("wonderboard","wonder board")
    s = s.replace("backerboard","backer board")
    s = s.replace("flatbraces","flat braces")
    s = s.replace("cieling","ceiling")
    s = s.replace("ceadar","cedar")
    s = s.replace("cedart","cedar")
    s = s.replace("frontload","front load")
    s = s.replace("stcking","sticking")
    s = s.replace("barreir","barrier")
    s = s.replace("ajustable","adjustable")
    s = s.replace("sinnk","sink")
    s = s.replace("pedelal","pedestal")
    s = s.replace("undermount","under mounted")
    s = s.replace("suppll","supple")
    s = s.replace("conditionerionar","conditioner")
    s = s.replace("vynal","vinyl")
    s = s.replace("aluminun","aluminum")
    s = s.replace("installbay","install bay")
    s = s.replace("cermic","ceramic")
    s = s.replace("plastice","plastic")
    s = s.replace("wattsolar","watt solar")
    s = s.replace("glaciar","glacier")
    s = s.replace("toliet","toilet")
    s = s.replace("garageescape","garage escape")
    s = s.replace("alumanam","aluminum")
    s = s.replace("treate","treated")
    s = s.replace("weathershield","weather shield")
    s = s.replace("conditionerioners","conditioner")
    s = s.replace("heaterconditioner","conditioner")
    s = s.replace("vbration","vibration")
    s = s.replace("fencde","fence")
    s = s.replace("knoty","knotty")
    s = s.replace("untility","utility")
    s = s.replace("christmass","christmas")
    s = s.replace("garlend","garland")
    s = s.replace("ceilig","ceiling")
    s = s.replace("glaciar","glacier")
    s = s.replace("dcanvas","canvas")
    s = s.replace("vaccum","vacuum")
    s = s.replace("garge","garage")
    s = s.replace("ridiing","riding")
    s = s.replace("barreir","barrier")
    s = s.replace("keorsene","kerosene")
    s = s.replace("lanterun","lantern")
    s = s.replace("infered","infrared")
    s = s.replace("hardiboard","hardboard")
    s = s.replace("keorsene","kerosene")
    s = s.replace("sinnk","sink")
    s = s.replace("pedelal","pedaled")
    s = s.replace("hindged","hinged")
    s = s.replace("bateries","batteries")
    s = s.replace("undercabinet","under cabinet")
    s = s.replace("ceilig","ceiling")
    s = s.replace("extention","extension")
    s = s.replace("firepits","fire pit")
    s = s.replace("edsel","edsal")
    s = s.replace("aire acondicionado","air conditioner")
    s = s.replace("linoliuml","linoleum")
    s = s.replace("hagchet","hatchet")
    s = s.replace("steele","steel")
    s = s.replace("dimable","dimmable")
    s = s.replace("lithum","lithium")
    s = s.replace("rayoby","ryobi")
    s = s.replace("washerparts","washer kit")
    s = s.replace("lituim","lithium")
    s = s.replace("naturlas","naturals")
    s = s.replace("softners","softener")
    s = s.replace("doorsmoocher","doors moocher")
    s = s.replace("sofn","soft")
    s = s.replace("scaleblaster","scale blaster")
    s = s.replace("pressue","pressure")
    s = s.replace("paito","patio")
    s = s.replace("mandare","mandara")
    s = s.replace("scod","cod")
    s = s.replace("ddummy","dummy")
    s = s.replace("florcant","floor cant")
    s = s.replace("prunning","pruning")
    s = s.replace("enrty","enrty")
    s = s.replace("outdoorfurniture","outdoor furniture")
    s = s.replace("handtools","hand tools")
    s = s.replace("treate","treated")
    s = s.replace("wheelbarow","wheelbarrow")
    s = s.replace("hhigh","high")
    s = s.replace("accordian","accordion")
    s = s.replace("preature","pressure")
    s = s.replace("steqamers","steamers")
    s = s.replace("onda pressure","honda pressure")
    s = s.replace("insallation","insulation")
    s = s.replace("contracor","multi color")
    s = s.replace("stell","steel")
    s = s.replace("sjhelf","shelf")
    s = s.replace("ridiing","riding")
    s = s.replace("drils","drills")
    s = s.replace("planel","panel")
    s = s.replace("robi","ryobi")
    s = s.replace("swivrl","swirl")
    s = s.replace("enrty","entry")
    s = s.replace("paneks","panels")
    s = s.replace("floo shets","flooring sheets")
    s = s.replace("gazhose","gas hose")
    s = s.replace("artifical","artifical")
    s = s.replace("insullation","insulation")
    s = s.replace("peper","peper")
    s = s.replace("extention","extension")
    s = s.replace("insulaion","insulation")
    s = s.replace("insullation","insulation")
    s = s.replace("unsulation","insulation")
    s = s.replace("upholstry","upholstery")
    s = s.replace("medicien","medicine")
    s = s.replace("floorinh","flooring")
    s = s.replace("heavyduty","heavy duty")
    s = s.replace("hardsware","hardware")
    s = s.replace("traiter","trailer")
    s = s.replace("bathroon","bathroom")
    s = s.replace("tsnkless","tankless")
    s = s.replace("shoplight","shop light")
    s = s.replace("consertrated","concentrated")
    s = s.replace("zeroturn","zero turn")
    s = s.replace("vynik","vinyl")
    s = s.replace("aircondiioner","air conditioner")
    s = s.replace("plexy glass","plastic sheet")
    s = s.replace("accesory","accessory")
    s = s.replace("koolaroo","coolaroo")
    s = s.replace("uplihght","uplight")
    s = s.replace("edsel","edsal")
    s = s.replace("outdooor","outdoor")
    s = s.replace("pivotangle","pivot angle")
    s = s.replace("plasticl","plastic")
    s = s.replace("varigated","variegated")
    s = s.replace("basemetnt","basement")
    s = s.replace("cornor","corner")
    s = s.replace("plaers","pliers")
    s = s.replace("soundfroofing","roofing underlayment")
    s = s.replace("storeage","storage")
    s = s.replace("fountin","fountain")
    s = s.replace("extention","extension")
    s = s.replace("polyeurethane","polyurethane")
    s = s.replace("plastice","plastic")
    s = s.replace("tilees","tiles")
    s = s.replace("byefold","bi fold")
    s = s.replace("whirpool","whirlpool")
    s = s.replace("diining","dining")
    s = s.replace("connnector","connector")
    s = s.replace("woodebn","wooden")
    s = s.replace("celliling","ceiling")
    s = s.replace("waterroo","water roof")
    s = s.replace("sofn","softener")
    s = s.replace("softners","softener")
    s = s.replace("shoipping","shopping")
    s = s.replace("trollie","trolly")
    s = s.replace("shoplight","shop light")
    s = s.replace("diining","dining")
    s = s.replace("elongagated","elongated")
    s = s.replace("sjhelf","shelf")
    s = s.replace("polycarbonite","polycarbonate")
    s = s.replace("koolaroo","coolaroo")
    s = s.replace("galvinized","galvanized")
    s = s.replace("bakewarte","bakeware")
    s = s.replace("kennal","kennel")
    s = s.replace("elongagated","elongated")
    s = s.replace("tolet","toilet")
    s = s.replace("aspiradora","vacuum")
    s = s.replace("aluminium","aluminum")
    s = s.replace("laminet","laminate")
    s = s.replace("elecronic","electronic")
    s = s.replace("dedwalt","dewalt")
    s = s.replace("vaccuum","vacuum")
    s = s.replace("diining","dining")
    s = s.replace("insided","inside")
    s = s.replace("towbehind","tow behind")
    s = s.replace("kidie","kidde")
    s = s.replace("batterys","battery")
    s = s.replace("nutru","nutri")
    s = s.replace("kitchenfaucet","kitchen faucet")
    s = s.replace("kitcheen","kitchen")

    s = s.replace("whirpool", "whirlpool")
    s = s.replace("whirlpoolga", "whirlpool")
    s = s.replace("whirlpoolstainless", "whirlpool stainless")
    
    # some title edits END?
    s = s.replace("&#39;s"," ")

    s = s.replace("  "," ")
    s = s.replace(",","") #could be number / segment later?
    s = s.replace("$"," ")
    s = s.replace("+"," plus ")
    s = s.replace(";"," ")
    s = s.replace(":"," ")
    s = s.replace("&amp;"," ")
    s = s.replace("&amp"," ")
    s = s.replace("?"," ")
    s = s.replace("-"," ")
    s = s.replace("#"," ")
    s = s.replace("("," ")
    s = s.replace(")"," ")
    s = s.replace("//","/")
    s = s.replace("..",".")
    s = s.replace(" / "," ovr ")
    s = s.replace(" \\ "," ")
    s = s.replace("."," . ")

    s = re.sub(r"(^\.|/)", r" ", s)
    s = re.sub(r"(\.|/)$", r" ", s)
    s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
    s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
    s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
    s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)

    s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
    s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in ", s)
    s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft ", s)
    s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb ", s)
    s = re.sub(r"([0-9]+)( *)(square|sq)\.?", r"\1sq ", s)
    s = re.sub(r"([0-9]+)( *)(cubic|cu)\.?", r"\1cu ", s)
    s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal ", s)
    s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz ", s)
    s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm ", s)
    s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm ", s)
    s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg ", s)
    s = re.sub(r"([0-9]+)( *)(volts|volt|v)\.?", r"\1volt ", s)
    s = re.sub(r"([0-9]+)( *)(watts|watt|w)\.?", r"\1watt ", s)
    s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp ", s)

    s = s.replace(" x ", " xby ")
    s = s.replace("*", " xby ")
    s = s.replace(" by ", " xby")
    s = s.replace("x0", " xby 0")
    s = s.replace("x1", " xby 1")
    s = s.replace("x2", " xby 2")
    s = s.replace("x3", " xby 3")
    s = s.replace("x4", " xby 4")
    s = s.replace("x5", " xby 5")
    s = s.replace("x6", " xby 6")
    s = s.replace("x7", " xby 7")
    s = s.replace("x8", " xby 8")
    s = s.replace("x9", " xby 9")
    s = s.replace("0x", "0 xby ")
    s = s.replace("1x", "1 xby ")
    s = s.replace("2x", "2 xby ")
    s = s.replace("3x", "3 xby ")
    s = s.replace("4x", "4 xby ")
    s = s.replace("5x", "5 xby ")
    s = s.replace("6x", "6 xby ")
    s = s.replace("7x", "7 xby ")
    s = s.replace("8x", "8 xby ")
    s = s.replace("9x", "9 xby ")

    s = s.replace("&"," ")
    s = s.replace("'"," ")
    s = s.replace("  "," ")
    s = s.replace(" . "," ")

    s = unidecode(s.lower())
    return [stemmer.stem(z) for z in s.split(" ")]

In [54]:
def str_common_word(list1, list2):
    str1 = ' '.join(list1)
    str2 = ' '.join(list2)
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt += 1
    return cnt

def str_whole_word(list1, list2, i_=0):
    cnt = 0
    str1 = ' '.join(list1)
    str2 = ' '.join(list2)
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [89]:
t0 = time()

df_all['search_term'] = df_all['search_term'].apply(str_stem)
df_all['product_title'] = df_all['product_title'].apply(str_stem)
df_all['product_description'] = df_all['product_description'].apply(str_stem)
df_all['brand_text'] = df_all['brand'].apply(str_stem)

print 'took %0.5fs.' % (time() - t0)

took 817.60057s.


In [91]:
t0 = time()


df_all['len_of_query'] = df_all['search_term'].apply(len)
df_all['len_of_title'] = df_all['product_title'].apply(len)
df_all['len_of_description'] = df_all['product_description'].apply(len)
df_all['len_of_brand'] = df_all['brand_text'].apply(len)

print 'took %0.5fs.' % (time() - t0)

took 0.19057s.


In [92]:
t0 = time()


df_all['query_in_title'] = \
        df_all.apply(lambda row: str_whole_word(row.search_term, row.product_title), axis=1)
df_all['query_in_description'] = \
        df_all.apply(lambda row: str_whole_word(row.search_term, row.product_description), axis=1)

df_all['word_in_title'] = \
        df_all.apply(lambda row: str_common_word(row.search_term, row.product_title), axis=1)
df_all['word_in_description'] = \
        df_all.apply(lambda row: str_common_word(row.search_term, row.product_description), axis=1)
df_all['word_in_brand'] = \
        df_all.apply(lambda row: str_common_word(row.search_term, row.brand_text), axis=1)

df_all['ratio_title'] = df_all['word_in_title'] / df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description'] / df_all['len_of_query']

print 'took %0.5fs.' % (time() - t0)

took 46.80548s.


In [93]:
df_all['ratio_brand'] = df_all['word_in_brand'] / df_all['len_of_brand'] 
df_all.loc[df_all.ratio_brand.isnull(), 'ratio_brand'] = 0.0
df_all.ratio_brand.isnull().sum()

0

In [94]:
df_all.brand = LabelEncoder().fit_transform(df_all.brand)


/home/agrigorev/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [95]:
df_all['search_term_length'] = df_all['search_term'].map(lambda x: len(' '.join(x)))

In [102]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

In [103]:
def identity_analyzer(l): return l

In [131]:
term_vec = TfidfVectorizer(analyzer=identity_analyzer)
X_terms = term_vec.fit_transform(df_all.search_term)

term_svd = TruncatedSVD(n_components=25, random_state=2016)
X_terms = term_svd.fit_transform(X_terms)

In [132]:
prod_title_vec = TfidfVectorizer(analyzer=identity_analyzer)
X_prod_title = term_vec.fit_transform(df_all.product_title)

prod_title_svd = TruncatedSVD(n_components=25, random_state=2016)
X_prod_title = prod_title_svd.fit_transform(X_prod_title)

In [134]:
prod_desc_vec = TfidfVectorizer(analyzer=identity_analyzer)
X_prod_desc = term_vec.fit_transform(df_all.product_description)

prod_desc_svd = TruncatedSVD(n_components=25, random_state=2016)
X_prod_desc = prod_desc_svd.fit_transform(X_prod_desc)

In [135]:
brand_vec = TfidfVectorizer(analyzer=identity_analyzer)
X_brand = term_vec.fit_transform(df_all.brand_text)

brand_svd = TruncatedSVD(n_components=25, random_state=2016)
X_brand = brand_svd.fit_transform(X_brand)

In [ ]:
product_desc = df_all.product_title + df_all.product_description + df_all.brand_text
queries = df_all.search_term

all_text = pd.concat((product_desc, queries))

In [121]:
all_vectorizer = TfidfVectorizer(analyzer=identity_analyzer)

all_text_svd = TruncatedSVD(n_components=70, random_state=2016)
all_text_svd.fit(all_vectorizer.fit_transform(all_text))

TruncatedSVD(algorithm='randomized', n_components=70, n_iter=5,
       random_state=2016, tol=0.0)

In [124]:
product_lsa = all_text_svd.transform(all_vectorizer.transform(product_desc))
queries_lsa = all_text_svd.transform(all_vectorizer.transform(queries))

lsa_similarity = np.sum(product_lsa * queries_lsa, axis=1)

In [126]:
columns = ['brand', 'len_of_query', 'len_of_title', 'len_of_description', 'len_of_brand',
           'query_in_title', 'query_in_description', 
           'word_in_title', 'word_in_description', 'word_in_brand',
           'ratio_title', 'ratio_description', 'ratio_brand',
           'search_term_length']

In [139]:
X_all = np.hstack([df_all[columns].values, X_terms, X_prod_desc, X_brand, lsa_similarity.reshape(-1, 1)])
X_all.shape

(240760, 90)

In [146]:
X = X_all[:len(df_train)]
y = df_train.relevance.values

X_test = X_all[len(df_train):]

In [156]:
train_len = int(len(df_train) * 0.885)
counts = df_train[:train_len].groupby(['product_uid']).count()[['id']]

counts = counts[counts['id'] > 1]
counts = counts.add_suffix('_Count').reset_index()
valid_product_uids = set(counts['product_uid'].values)

inds = []

allowed_uids = df_train.loc[df_train['product_uid'].isin(valid_product_uids)]
# For now, always grab first row of valid product uid.
lastUid = 0

for idx, mrow in allowed_uids.iterrows():
    if lastUid == mrow['product_uid']:
        continue

    lastUid = mrow['product_uid']
    inds.append(idx)

validation_idx = np.array(inds + list(df_train[train_len:].index.values))
train_idx = np.array(df_train.loc[~df_train.index.isin(validation_idx)].index.values)

In [160]:
X_train = X[train_idx]
y_train = y[train_idx]
X_val = X[validation_idx]
y_val = y[validation_idx]

In [164]:
from sklearn.cross_validation import ShuffleSplit, KFold

In [167]:
cv = KFold(n=X.shape[0], n_folds=5, shuffle=True, random_state=0x31337)

scores = []
for train, test in cv:
    rfr = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=2016, max_features=10, max_depth=20, 
                            verbose=1)
    
    rfr.fit(X[train], y[train])
    y_pred = rfr.predict(X[test])
    score = fmean_squared_error(y[test], y_pred)
    print score
    
    scores.append(score)

np.mean(scores), np.std(scores)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   36.6s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished


0.45655760404


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   36.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished


0.457425275563


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   36.7s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished


...

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   36.4s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished


0.451459622042


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   36.7s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished


0.453166083572


(0.45440669836897712, 0.0022327786480899722)

In [162]:
y_valid_pred = rfr.predict(X_val)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.4s finished


In [163]:
fmean_squared_error(y_val, y_valid_pred)

0.46484409556949735

In [150]:
y_score = rfr.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    3.3s finished


In [151]:
res = pd.DataFrame({"id": df_test.id, "relevance": y_score})
res.to_csv('submission.csv', index=False)